In [7]:
from module.file_utils import *
from module.camera import *
from module.pose import *
import os

import numpy as np
from module.hybrid_math import transpose2d
# pose parsing 6 dof to 4*4 extrinsic

# seq, rot, translation, timestamp
# garage
cam2rigs = [] # 0,1,2,3
cam2rigs.append([0.0, -0.785398156308, 3.4331e-08, -0.20000000298, 8.742e-09,
        0.20000000298])
cam2rigs.append([0.0, 0.785398156308, -3.4331e-08, 0.20000000298, 8.742e-09, 0.20000000298])
cam2rigs.append([0.0, 2.356194497281, -1.02993e-07, 0.20000000298, -8.742e-09,
        -0.20000000298])
cam2rigs.append([0.0, -2.356194497281, 1.02993e-07, -0.20000000298, -8.742e-09,
        -0.20000000298])
# train pose
r2ws = np.loadtxt("./train_trajectory.txt")

def get_rot_vec_t(rot_vec_t):
    rot_vec = rot_vec_t[0:3]
    tr = rot_vec_t[3:]
    return Pose.from_rot_vec_t(np.array(rot_vec),np.array(tr))


for cam_id,cam2rig in enumerate(cam2rigs):
    c2r = get_rot_vec_t(cam2rig)
    for idx,r2w in enumerate(r2ws):
        r2w = get_rot_vec_t(r2w[1:-1])
        mat4 = r2w.mat44()@c2r.mat44()
        np.savetxt(f"./garage_pinhole/pose/0_train_{cam_id*20 + idx:04d}.txt", mat4)






In [ ]:
# image rename
from module.file_utils import read_image 
for cam_id in range(4):
    for idx in range(20):
        image = read_image(f"./garage_nerf_pinhole/cam{cam_id+1}_square_640/{idx+1:05d}.png")
        write_image(image, f"./garage_pinhole/rgb/0_train_{cam_id*20 + idx:04d}.png")






In [23]:
from module.file_utils import *
from module.plot import *

def read_float_test(path: str) -> np.ndarray:
    extension = path.split(sep=".")[-1]
    try:
        if extension == "npy":
            data = np.load(path)
        elif extension == 'tiff':
            multi_datas = tifffile.TiffFile(path)
            num_datas = len(multi_datas.pages)
            if num_datas == 0: raise Exception("No Images.")
            elif num_datas == 1: data = multi_datas.pages[0].asarray().squeeze()
            elif num_datas == 2: data = (multi_datas.pages[0].asarray().squeeze(),multi_datas.pages[1].asarray().squeeze()) # with thumbnail
            else: data = concat([expand_dim(x.asarray(),0) for x in multi_datas.pages], 0)
        else:
            raise Exception("No Support Extension.")
    except Exception as e:
        print("reading data failed.")
        print(e)
        data = None
    return data


for cam_id in range(4):
    for idx in range(20):
        _, depth = read_float_test(f"./garage_nerf_pinhole/cam{cam_id+1}_square_640_depth/{idx+1:05d}.tiff")
        depth = np.clip(1./ depth, 0., 20.)
        write_float(depth, f"./garage_pinhole/depth/0_train_{cam_id*20 + idx:04d}.tiff")
        depth_image = float_to_image(depth,0.,10.)
        write_image(depth_image, f"./garage_pinhole/depth_vis/0_train_{cam_id*20 + idx:04d}.png")


In [1]:
from module.file_utils import *
from module.camera import *
from module.pose import *
import os

import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

depth = read_float("./garage_pinhole/depth/0_train_0000.tiff")
print(depth)


[[3.1376033  3.1391048  3.1406078  ... 3.5794125  3.5769482  3.5723984 ]
 [3.14736    3.148871   3.1503832  ... 3.581509   3.576948   3.5723982 ]
 [3.1571774  3.1586974  3.1602194  ... 3.5815086  3.576948   3.5723977 ]
 ...
 [2.14903    2.145895   2.142769   ... 0.9441197  0.94411904 0.9441182 ]
 [2.1490288  2.1458938  2.142768   ... 0.9401613  0.94016033 0.9401596 ]
 [2.1490273  2.1458924  2.1427665  ... 0.9362355  0.9362347  0.9362339 ]]


In [15]:
# FOV = 60

fx =  640. / (2*np.tan(np.pi / 6))
fy =  480. / (2*np.tan(np.pi / 6))
print(fx)
print(fy)

554.2562584220408
415.69219381653056


In [20]:
# verification of the result
# Back-Projection
import numpy as np
import os.path as osp
frame_dict = {}
root = "./garage_pinhole"
frame_dict["root"] = root
frames = []


cam = {
    "cam_type": "PINHOLE",
    "image_size": [
     640,
     480
    ],
    "focal_length": [
     554.256,
     415.692
    ],
    "principal_point": [
     320,
     240
    ],
    "skew": 0.,
    "radial": [
     0.0,
     0.0,
     0.0
    ],
    "tangential": [
     0.0,
     0.0
    ]
   }


for i in range(2):
    frame = {}
    # camera
    frame["cam"] = cam
    # pose
    pose = np.loadtxt(osp.join(root,f"pose/0_train_{i:04d}.txt"))
    frame["camtoworld"] = pose
    frame["image"] = f"rgb/0_train_{i:04d}.png"
    frame["depth"] = f"depth/0_train_{i:04d}.tiff"
    frames.append(frame)

frame_dict["frames"] = frames 




In [21]:
from module.geometry import *

multiview =  MultiView.from_dict(frame_dict)

multiview.save_point_cloud("./pcd.ply")